In [1]:
import numpy as np
from sentence_transformers import SentenceTransformer
from google import genai
import json
from dotenv import load_dotenv

c:\Users\Koustav Chatterjee\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load environmental variables

load_dotenv()

True

In [3]:
# embedding model

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [4]:
client = genai.Client()

In [5]:
# cosine similarity search


def cosine_similarity(a, b):
    dot_prod = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)

    if norm_a == 0 or norm_b == 0:
        return 0

    return dot_prod / (norm_a * norm_b)

In [6]:
def retrieve(query, embeddings, k=3):
    query_emb = model.encode(query)

    scores = []
    for emb in embeddings:
        score = cosine_similarity(query_emb, np.array(emb["embedding"]))

        scores.append((score, emb))

    scores.sort(reverse=True, key=lambda x: x[0])
    return [ing for _, ing in scores[:k]]

In [10]:
# test

with open("embeddings.json", "r", encoding="utf-8") as f:
    embeddings = json.load(f)

user_query = "What is a stack?"

retrieved = retrieve(user_query, embeddings, k=3)

context = "\n\n".join(c["content"] for c in retrieved)

prompt = f"""
You are an expert study helper and remember everything the user has studied.
Give answer to the user query based on the provided context only.

User Query:
{user_query}

Context:
{context}

Answer in simple english to the user.
"""

response = client.models.generate_content(model="gemini-2.5-flash", contents=prompt)

print(response.text)

A stack is an Abstract Data Type (ADT) that behaves like a real-world stack, such as a deck of cards or a pile of plates. It allows operations (like placing or removing items) to occur only at one end, which is always referred to as the "top" of the stack.

This characteristic makes it a **LIFO** (Last-In-First-Out) data structure, meaning the last element added to the stack is the first one to be removed.

Key operations include:
*   **push()**: Adding an element to the top of the stack.
*   **pop()**: Removing an element from the top of the stack.

There are also other useful functions like:
*   **peek()**: To look at the top element without removing it.
*   **isFull()**: To check if the stack is full.
*   **isEmpty()**: To check if the stack is empty.

A pointer, called 'top', always keeps track of the last element pushed onto the stack. Stacks can be of fixed size or dynamically resize, and they can be implemented using arrays for a fixed-size version.
